In [1]:
"""
Copyright (c) Meta Platforms, Inc. and affiliates.
All rights reserved.

This source code is licensed under the license found in the
LICENSE file in the root directory of this source tree.
"""

from pathlib import Path

from jmp.configs.pretrain.jmp_l import jmp_l_pt_config_
from jmp.tasks.pretrain import PretrainConfig, PretrainModel
from jmp.tasks.pretrain.module import (
    NormalizationConfig,
    PretrainDatasetConfig,
    TaskConfig,
)


# Let's make the config
def jmp_l_config():
    config = PretrainConfig.draft()

    jmp_l_pt_config_(config)

    # Set data config
    config.batch_size = 4
    config.num_workers = 8

    # Set the tasks
    config.tasks = [
        # TaskConfig(
        #     name="oc20",
        #     train_dataset=PretrainDatasetConfig(
        #         src=Path("/datasets/s2ef/2M/train/"),
        #         metadata_path=Path("/datasets/s2ef/2M/train_metadata.npz"),
        #     ),
        #     val_dataset=PretrainDatasetConfig(
        #         src=Path("/datasets/s2ef/all/val_id/"),
        #         metadata_path=Path("/datasets/s2ef/all/val_id_metadata.npz"),
        #     ),
        #     energy_loss_scale=1.0,
        #     force_loss_scale=73.0,
        #     normalization={
        #         "y": NormalizationConfig(mean=0.0, std=24.901469505465872),
        #         "force": NormalizationConfig(mean=0.0, std=0.5111534595489502),
        #     },
        # ),
        # TaskConfig(
        #     name="oc22",
        #     train_dataset=PretrainDatasetConfig(
        #         src=Path("/shared/pre-training-datasets/oc22/s2ef-total/train/"),
        #     ),
        #     val_dataset=PretrainDatasetConfig(
        #         src=Path("/shared/pre-training-datasets/oc22/s2ef-total/val_id/"),
        #     ),
        #     energy_loss_scale=1.0,
        #     force_loss_scale=80.0,
        #     normalization={
        #         "y": NormalizationConfig(mean=0.0, std=25.229595396538468),
        #         "force": NormalizationConfig(mean=0.0, std=0.25678861141204834),
        #     },
        # ),
        # TaskConfig(
        #     name="ani1x",
        #     train_dataset=PretrainDatasetConfig(
        #         src=Path("/shared/pre-training-datasets/ani1x/train/"),
        #     ),
        #     val_dataset=PretrainDatasetConfig(
        #         src=Path("/shared/pre-training-datasets/ani1x/val/"),
        #     ),
        #     energy_loss_scale=1.0,
        #     force_loss_scale=15.0,
        #     normalization={
        #         "y": NormalizationConfig(mean=0.0, std=2.8700712783472118),
        #         "force": NormalizationConfig(mean=0.0, std=2.131422996520996),
        #     },
        # ),
        TaskConfig(
            name="transition1x",
            train_dataset=PretrainDatasetConfig(
                src=Path("/shared/pre-training-datasets/trans1x/train/"),
            ),
            val_dataset=PretrainDatasetConfig(
                src=Path("/shared/pre-training-datasets/trans1x/val/"),
            ),
            energy_loss_scale=1.0,
            force_loss_scale=14.0,
            normalization={
                "y": NormalizationConfig(mean=0.0, std=1.787466168382901),
                "force": NormalizationConfig(mean=0.0, std=0.3591422140598297),
            },
        ),
    ]

    return config.finalize()


config = jmp_l_config()
print(config)

configs: list[tuple[PretrainConfig, type[PretrainModel]]] = []
configs.append((config, PretrainModel))

id='sqjvgggl' trainer=TrainerConfig(optimizer=OptimizerConfig(log_grad_norm=True, gradient_clipping=GradientClippingConfig(value=1.0)), supports_skip_batch_exception=False, supports_parameter_hooks=False, set_float32_matmul_precision='medium', precision='16-mixed', use_distributed_sampler=False) optimizer=AdamWConfig(lr=0.0003, weight_decay=0.1, betas=(0.9, 0.95)) lr_scheduler=LinearWarmupCosineAnnealingSchedulerConfig(warmup_steps=2000, max_epochs=2, warmup_start_lr_factor=0.2, min_lr_factor=0.1) edge_dropout=0.1 backbone=BackboneConfig(num_spherical=7, num_radial=128, num_blocks=6, emb_size_atom=256, emb_size_edge=1024, emb_size_trip_in=64, emb_size_trip_out=128, emb_size_quad_in=64, emb_size_quad_out=32, emb_size_aint_in=64, emb_size_aint_out=64, emb_size_rbf=32, emb_size_cbf=16, emb_size_sbf=64, num_before_skip=2, num_after_skip=2, num_concat=4, num_atom=3, num_output_afteratom=3, num_atom_emb_layers=2, direct_forces=True, sbf={'name': 'legendre_outer'}, quad_interaction=True, atom

/home/sanjeevr/MLFF-distill/JMP/src/jmp/lightning/model/config.py:718: BaseConfig._rng is None. The generated IDs will not be reproducible. To fix this, call BaseConfig.set_seed(...) before generating any IDs.


In [2]:
from jmp.lightning import Runner, Trainer

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def run(config: PretrainConfig, model_cls: type[PretrainModel]) -> None:
    model = model_cls(config)
    trainer = Trainer(config)
    trainer.fit(model)


runner = Runner(run)
runner.fast_dev_run(configs, n_batches=16)

Fast dev run:   0%|          | 0/1 [00:00<?, ?it/s]

Failed to import lovely-tensors. Ignoring pretty PyTorch tensor formatting
Failed to import rich. Falling back to default Python logging.
CRITICAL:jmp.lightning.trainer.trainer:Setting config.trainer.default_root_dir='/home/sanjeevr/MLFF-distill/JMP/config/lightning_logs/89nm5lk0'.
Seed set to 0
CRITICAL:jmp.lightning.util.seed:Set global seed to 0.
CRITICAL:jmp.lightning.runner:Auto-wrapping run in Trainer context


Unrecognized arguments:  dict_keys(['learnable_rbf', 'learnable_rbf_stds', 'unique_basis_per_layer', 'dropout', 'edge_dropout'])
> /tmp/ipykernel_3045647/744631729.py(9)run()
      7     model = model_cls(config)
      8     import pdb; pdb.set_trace()
----> 9     trainer = Trainer(config)
     10     trainer.fit(model)
     11 

220879104


CRITICAL:jmp.lightning.trainer.trainer:Ran 0 finalizers for Trainer cleanup.
CRITICAL:jmp.lightning.util.seed:Reset global seed.
CRITICAL:jmp.lightning.runner:Error in run with run_id='sqjvgggl' (run_name=None): 
Traceback (most recent call last):
  File "/home/sanjeevr/MLFF-distill/JMP/src/jmp/lightning/runner.py", line 456, in fast_dev_run
    return_value = self.local((config, *args), env=env, reset_id=True)
  File "/home/sanjeevr/.local/lib/python3.10/site-packages/typing_extensions.py", line 2853, in wrapper
    return arg(*args, **kwargs)
  File "/home/sanjeevr/MLFF-distill/JMP/src/jmp/lightning/runner.py", line 198, in local
    return_value = self._run_fn(config, *args)
  File "/home/sanjeevr/MLFF-distill/JMP/src/jmp/lightning/runner.py", line 120, in wrapped_run
    return run(config, *args)
  File "/tmp/ipykernel_3045647/744631729.py", line 9, in run
    trainer = Trainer(config)
  File "/tmp/ipykernel_3045647/744631729.py", line 9, in run
    trainer = Trainer(config)
  File